In [38]:
from ham1d.models.bitmanip import select_states
from ham1d.models import bitmanip as bmp

from ham1d.operators.spin1d import _construct_ops 
#from ham1d.operators.spin1d._construct_ops import _ham_ops
#from ham1d.models.buildham import buildham

from scipy.sparse import csr_matrix
from ham1d.models import ferm1d as fe1


import numpy as np
import numba as nb
from numba import njit, prange

#from ham1d.operators.ferm1d import _fermops as fo
# ---------------------------------------------------------
# STATE SELECTION HELPER ROUTINES


@nb.njit("uint32(uint64)")
def countSetbits(n):
    """
    A routine that counts the
    number of set bits in a given
    number (state) -> states in our
    code are encoded as numbers where
    we make use of the mapping to the
    binarry representation.

    INPUT:

    n: uint64
        A number(state) for which the
        number of set bits is to be
        determined.

    OUTPUT:

    count: uint32
        The number of set bits.

    """

    count = np.uint32(0)

    while(n):

        #  bitwise AND -> checks
        #  the rightmost bit
        count += n & 1
        #  right shift until
        #  there is nothing left
        #  to shift
        n >>= 1

    return count


@nb.njit('Tuple((uint64[:], uint64[:]))(uint32, uint32)', nogil=True)
def select_states(L, nup):
    """
    A routine for selecting states from a block
    with a given number of nup spins or, in the
    fermionic language, a given number of particles.
    This is only applicable to models which
    preserve the particle number/spin projection
    along the z-axis.

    INPUT:

    L: uint32
        The length of the 1-dimensional chain.

    nup: uint32
        The number of up-spins or particles.

    OUTPUT:

    states: np.array, dtype=uint64
        An array of appropriate states selected
        from the whole 2^L dimensional Hilbert
        space of the problem
    state_indices: np.array, dtype=uint64
        A matrix containing information on how
        the selected states from the entire Hilbert
        space are ordered in the selected nup
        subspace of the Hilbert space.

    """
    states = []
    state_indices = np.zeros(1 << L, dtype=np.uint64)
    idx = 0
    
    nsteps = 1 << L 
    for i in range(nsteps):

        if countSetbits(i) == nup:

            states.append(i)
            state_indices[i] = idx

            idx += 1

    return np.array(states, dtype=np.uint64), state_indices



@nb.njit("Tuple((float64, uint64))(uint64, uint32)", fastmath=True)
def cn(state, bit):
    """
    The number operator operator, in matrix form given as:

    cn = 0.5 * np.array([[1, 0], [0, -1]])

    Note that in our implementation, we write the cn
    operator in a particle-hole symmetric way, applying
    the transformation n -> n - 1/2
    """

    return 0.5 * (-1) ** bmp.getbit(state, bit), state


@nb.njit("Tuple((float64, uint64))(uint64, uint32)", fastmath=True)
def id2(state, bit):
    """
    The identity operator.
    """

    return 1., state


@nb.njit("Tuple((float64, uint64))(uint64, uint32)", fastmath=True)
def cp(state, bit):
    """
    creation operator, in matrix form given as:

    cp = np.array([[0, 1], [0, 0]])
    """

    bitval = bmp.getbit(state, bit)

    if bitval:

        return 1. - bitval, state

    else:
        return 1. - bitval, bmp.bitflip(state, bit)


@nb.njit("Tuple((float64, uint64))(uint64, uint32)", fastmath=True)
def cm(state, bit):
    """
    annihilation operator, in matrix form given as:

    cm = np.array([[0, 0], [1, 0]])
    """

    bitval = bmp.getbit(state, bit) * 1.

    if bitval:

        return bitval, bmp.bitflip(state, bit)

    else:

        return bitval, state
    
signature = (
    'Tuple((uint64[:], uint64[:], float64[:]))(uint64[:], uint64[:], float64[:], uint32[:,:], uint32[:])')


@nb.njit(signature, fastmath=True, nogil=True, cache=True)
def _ham_ops(states, state_indices, couplings, sites, sel_opt):
    """
    Numba-optimized code for hamiltonian construction in the
    1D fermionic case for arbitrary couplings between (arbitrarily
    chosen) sites. This routine is internal and is intended
    for usage in the operators.buildham.buildham(...) function.
    The output of this function are the matrix elements
    corresponding to a single hamiltonian term in the decomposition
    H = \sum_i h_i
    where H is the entire hamiltonian and h_i are the aforementioned
    subterms.

    INPUT:

    states: np.array, dtype=np.uint64
        An array of states available to the system.
    state_indices: np.array, dtype=np.uint64
        A helper array used for finding the index
        of a state which was obtained after hamiltonian
        action on some selected initial state. Both
        states and state_indices arrays are usually
        provided as the output of the
        bitmanip.select_states(...) function.
    couplings: np.array, dtype=np.complex128
        An array of the coupling constants which
        are simply (complex) multiplicative
        prefactors for the hamiltonian matrix
        elements.
    sites: 2D np.array, dtype=np.uint32,
           shape=(len(couplings), len(sel_opt))
        An array indicating which sites the
        operators specified in the sel_opt (see below)
        and their corresponding coupling constants
        (see 'couplings' above) should couple.
    sel_opt: np.array, dtype=uint32
        An array specifying which operators constitute
        a particular hamiltonian term. Allowed entries
        in the sel_opt array are:

            0: c^+ operator
            1: c^- operator
            2: Identity operator
            3: n operator (c+c-)


    OUTPUT:

    rows, cols: np.array(s), dtype=np.uint64
        Rows and columns arrays needed in the
        construction of a sparse array.
    vals: np.array, dtype=np.complex128
        Values of the matrix elements whose
        positions are given by the entries in
        the rows and cols arrays.

    EXAMPLE:

        Here we show how to construct individual
        hamiltonian terms the 1D Anderson
        hamiltonian:

        H_a = t * (\sum_i (c_i^+c_{i+1}^- +
                 + c_{i+1}^+ c_{i+1}^-) +
                 + \sum_i h_i n_i

        We set L=4 with PBC in the half-filled
        case, nup=2. We set t=-1 and the
        disorder strength parameter W=1.

        First, the selection of states is
        most conveniently done as follows:

        states, state_indices = bmp.select_states(L=4, nup=2)

        We define the coupling constants:

        t = -1
        W = 1.

        Let us simulate hopping to the right first:

            couplings_r = np.array([t for i in range(L)])
            sites_r = np.array([[i, (i + 1) % L] for i in range(L) ]) #PBC
            sel_opt_r = [0, 1]

        Hopping to the left:

            couplings_l = np.array([t for i in range(L)])
            sites_l = np.array([[i % L, (i - 1) % L] for i in range(L) ]) #PBC
            sel_opt_l = [0, 1]

        Let us now define the random fields in the random part:

            couplings_rnd = np.random.uniform(-W, W, size=L)
            sites_rnd = np.array([i for i in range(L)])
            sel_opt_rnd = [3]

        To construct any of the above terms, simply call:

            _ham_ops(states, state_indices, couplings_{}, sites_{},
                     sel_opt_{})

        NOTE: as it is evident from the above cases, the sites
        array's shape should be of the shape (len(couplings), len(sel_opt)).

    """

    sel_opt = sel_opt[::-1]

    rows = []
    cols = []
    vals = []

    # for i, state in enumerate(states):
    for i in range(len(states)):
        state = states[i]
        for j, coupling in enumerate(couplings):

            sitelist = sites[j][::-1]

            newstate = state
            factor = 1.

            for k, site in enumerate(sitelist):

                if sel_opt[k] == 0:  # c+

                    # Consider the fermionic sign
                    ferm_sgn = (-1)**bmp.countBitsInterval(newstate,
                                                           0, site)

                    factor_, newstate = cp(newstate, site)

                    factor_ *= ferm_sgn

                elif sel_opt[k] == 1:  # c-

                    # consider the fermionic sign
                    ferm_sgn = (-1)**bmp.countBitsInterval(newstate,
                                                           0, site)

                    factor_, newstate = cm(newstate, site)

                    factor_ *= ferm_sgn

                elif sel_opt[k] == 2:  # identity

                    factor_, newstate = id2(newstate, site)

                elif sel_opt[k] == 3:  # number operator

                    factor_, newstate = cn(newstate, site)

                factor *= factor_

            if factor:

                rows.append(i)
                cols.append(state_indices[newstate])
                vals.append(coupling * factor)

    rows = np.array(rows, dtype=np.uint64)
    cols = np.array(cols, dtype=np.uint64)
    vals = np.array(vals, dtype=np.float64)

    return rows, cols, vals


_trans_dict = {'+': 0,
               '-': 1,
               'I': 2,
               'n': 3}

operators = {'+': cp,
             '-': cm,
             'I': id2,
             'n': cn}





def buildham(L, nup, static_list):

    states, state_indices = select_states(np.uint32(L), np.uint32(nup))
    states = np.uint64(states)
    state_indices = np.uint64(state_indices)
    nstates = states.shape[0]
    ham = csr_matrix((nstates, nstates), dtype=np.float64)
    ham = 0
    trans_dict = _trans_dict
    ham_ops = _ham_ops
    
    for term in static_list:

        ops = list(term[0])
        ops = np.array([trans_dict[op] for op in ops], dtype=np.uint32)

        coupsites = np.array(term[1])

        coups = np.float64(coupsites[:, 0])

        sites = np.uint32(coupsites[:, 1:])
        rows, cols, vals = ham_ops(states, state_indices, coups, sites, ops)
        ham += csr_matrix((vals, (rows, cols)), shape=(nstates, nstates))
    
    return ham


In [31]:
import os
os.environ['NUMBA_DISABLE_JIT'] = "0"

L=22
nu=11
J= np.sqrt(2)
W=1.
Delta = 1.0
gldn = (np.sqrt(5.) - 1.) * 0.5
%timeit states, state_indices = select_states(L, nu)
couplings = np.ones(L, dtype=np.float64)
sites = np.array([[i, (i+1)%L] for i in range(L)], dtype=np.uint32)

sel_opt = np.array([0, 1], dtype=np.uint32)

75.2 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [137]:
_ham_ops(states, state_indices, couplings, sites, sel_opt)

(array([     0,      1,      1, ..., 705430, 705430, 705431], dtype=uint64),
 array([352726,      0, 352736, ..., 705429, 705419, 705430], dtype=uint64),
 array([1., 1., 1., ..., 1., 1., 1.]))

In [138]:
states.shape

(705432,)

In [32]:
J_pm = [[J * 0.5 , i, (i + 1)%L] for i in range(L-1)] # the %L part ensures PBC
J_mp = [[J * 0.5 , (i+1)%L, i] for i in range(L-1)]
# the first entry in the nested list is always the value of the exchange constant
# the following entries are integers specifying the sites on which the operators act.

# then, specify the operators:
flip_left = ['+-', J_pm]

# the first entry is the operator specification string (opstring), the second one is the site coupling
# list. len(list(opstring)) should match the number of sites in the site coupling list, otherwise the
# could will raise an error.
# The allowed operators in the spin case
# are:
# {'+': s+, '-': s-, 'I': id2, 'x': sx, 'y': sy, 'z': sz}
 
# the specification of operators together with the site-coupling list J_pm completely
# determines one single hamiltonian term. 

# Let's now define the other hamiltonian terms, namely the right-flipping term, the interaction
# term and the disorder term.

# right-flipping:
flip_right = ['+-', J_mp] #couplings remain the same, only the operator order changes

# interaction:
J_zz = [[Delta, i, (i+1)%L] for i in range(L-1)]
inter = ['nn', J_zz]

#random field

fields = np.ones(L) / np.sqrt(3) #np.array([W * np.cos(2*np.pi*gldn*i) for i in range(1, L+1)])
J_z = [[fields[i], i] for i in range(L)]
rnd = ['n', J_z]

# we can now put together the static_list -> just a list of hamiltonian term
# definitions

static_list = [flip_left, flip_right, inter, rnd]





%time buildham(L, nu, static_list, _construct_ops)



In [33]:
%timeit buildham(L, nu, static_list)

6.78 s ± 722 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [126]:
%time ham = sp1.hamiltonian(L, static_list, [], Nu=nu,parallel=False, mpisize=0, mpirank=0)

#ham._static_changed=True
#%time ham.build_mat()

Please wait, building the Hamiltonian ...
Building the Hamiltonian finished!
Calculating nnz, o_nnz, d_nnz!
Calculating nnz, o_nnz, d_nnz finished!
CPU times: user 6.31 s, sys: 1.1 s, total: 7.41 s
Wall time: 6.98 s


In [52]:
%timeit select_states(18,9)

4.38 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
select_states.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function select_states, <ipython-
input-40-34add9b8fe9f> (49)  


Parallel loop listing for  Function select_states, <ipython-input-40-34add9b8fe9f> (49) 
----------------------------------------------------------------------------------------|loop #ID
@nb.njit('Tuple((uint64[:], uint64[:]))(uint32, uint32)', parallel=True, nogil=True)    | 
def select_states(L, nup):                                                              | 
    """                                                                                 | 
    A routine for selecting states from a block                                         | 
    with a given number of nup spins or, in the                                         | 
    fermionic language, a given number of particles.                                    | 
    This is only applicable to models which                                             | 
    preserve the particle number/spin projection                         

In [32]:
_ham_ops.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function _ham_ops, <ipython-
input-31-8ecdce37b75f> (101)  


Parallel loop listing for  Function _ham_ops, <ipython-input-31-8ecdce37b75f> (101) 
--------------------------------------------------------------------------------|loop #ID
@nb.njit(_signature, fastmath=True, nogil=True, cache=True, parallel=True)      | 
def _ham_ops(states, state_indices, couplings, sites, sel_opt):                 | 
    """                                                                         | 
    Numba-optimized code for hamiltonian construction in the                    | 
    spin 1/2 case for arbitrary couplings between (arbitrarily                  | 
    chosen) sites. This routine is internal and is intended                     | 
    for usage in the operators.buildham.buildham(...) function.                 | 
    The output of this function are the matrix elements                         | 
    corresponding to a single hamiltonian term in the decompos

In [88]:
nb.config.NUMBA_NUM_THREADS=1

In [3]:
@nb.njit(fastmath=True, parallel=True)
def test_numba(a):
    
    val = 0.
    for i in prange(a.shape[0]):
        val += a[i]**2
        
    return val

In [ ]:
test_numba()

In [4]:
input_ndarray = np.arange(1., 100000., 1.)

In [6]:
%timeit test_numba(input_ndarray)

16.6 µs ± 1.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [79]:
from numba import autojit

@nb.njit()
def do_sum(A):
    acc = 0.0
    for i in range(A.shape[0]):
        acc += np.sqrt(A[i])

    return acc


@njit(parallel=True)
def do_sum_parallel(A):
    # each thread can accumulate its own partial sum, and then a cross
    # thread reduction is performed to obtain the result to return
    n = len(A)
    acc = 0.
    for i in prange(n):
        acc += np.sqrt(A[i])
    return acc

@njit(parallel=True, fastmath=True)
def do_sum_parallel_fast(A):
    n = len(A)
    acc = 0.
    for i in prange(n):
        acc += np.sqrt(A[i])
    return acc

In [81]:
%timeit do_sum(input_ndarray)



244 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [83]:
%timeit do_sum_parallel(input_ndarray)

149 µs ± 9.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [84]:
%timeit do_sum_parallel_fast(input_ndarray)

84.2 µs ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
import numpy as np
import numba as nb

LOOKUP_TABLE = np.array([
    1, 1, 2, 6, 24, 120, 720, 5040, 40320,
    362880, 3628800, 39916800, 479001600,
    6227020800, 87178291200, 1307674368000,
    20922789888000, 355687428096000, 6402373705728000,
    121645100408832000, 2432902008176640000], dtype='int64')


@nb.njit('i8(i8,i8)')
def binomial(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke.
    See http://stackoverflow.com/questions/3025162/statistics-combinations-in-python
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0
    
@nb.njit(fastmath=True, parallel=True)
def _ind0(iconf, npart):
    """
    A routine that finds a configuration index for
    a lattice with only two entities (spins and holes
    or up and down spins, for instance). NOTE:
    This is an internal (helper) routine, that is
    meant to be called from the ind routine.

    Parameters:
    -----------
    iconf - a configuration array, entries
            correspond to the sites at which
            the entities reside. For the
            following spin configuration:
            u d d u
            the iconf array would be:
            0 3 0 0
    npart - number of particles on a lattice

    Returns:
    --------

    indx: int
          The index of the state with a given
          configuration of up spins/particles.

    """
    indx = 1

    for i in prange(0, npart):

        # we obtain the site at which
        # the particle resides
        site = iconf[i]

        # we add the number of possible
        # preceeding combinations
        # i + 1 ensures proper increase
        indx += binomial(site, i + 1, )

    return indx


@nb.njit(fastmath=True, parallel=True)
def ind(iconf, L, nu):
    """
    A routine that finds a configuration index for a
    lattice with a
    given configuration of holes and up, down spins.

    INPUT:
    iconf - configuration array
    L - lattice size
    nu - number of up spins
    """

    icu = np.zeros(L, dtype=np.int64)
    # jh = 0
    ju = 0
    for i in range(L):

        if iconf[i] == 1:
            icu[ju] = i
            ju += 1

    # find indexes of the hole and up spin configurations:
    iu = _ind0(icu, nu)

    ind = iu

    return ind


@nb.njit()
def _conf0(ind, L, npart):
    """
    A routine to find a given configuration index for
    the case when there are only two entities on a lattice
    (for instance electrons/holes, up/down spins)

    Parameters:
    -----------

    ind: int
         configuration index of a requested state.
    L: int
       system size.
    npart: int
           Number of entities, such as particles or
           up pointing spins.

    Returns:
    --------

    iconf: ndarray, int
           A configuration array specifying at which
           sites the entities reside. Example for
           the following spin configuration:
           u d d u
           iconf => 0 3 0 0

    """

    if ind < 1:

        # print("Indexing should start with 1.")

        return

    ind -= 1
    iconf = np.zeros(L, dtype=np.int64)

    isite = L - 1
    while npart > 0:
        if ind >= binomial(isite, npart, ):
            iconf[npart - 1] = isite
            ind -= binomial(isite, npart, )

            npart -= 1

        isite -= 1

    return iconf


@nb.njit()
def conf(ind, L, nu, nh=0):
    if ind < 1:
        # print("Indexing starts with 1.")
        return

    iconf = np.zeros(L, dtype=np.int64)
    # iconfh=np.zeros(L, dtype=int)
    # iconfu=np.zeros(L, dtype=int)
    # the number of spin up configurations:

    iconfu = _conf0(ind, L, nu)

    for i in range(nu):
        iconf[iconfu[i]] = 1

    return iconf

In [53]:
%timeit val=_ind0(np.array([0,3,0,0]), 2)

#print(val)

1.76 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
